## DBCSAN Clustering
`另一种聚类方式：将所有数字替换为0，不经过分词直接聚类`
``` python
re.sub(r'\d+(\.\d+)?', '0', text)
```

In [ ]:
import pandas as pd
from utils.cluster import reassign_clusters, cluster, vectorize, tokenize,Cluster

datasets = ['BGL', 'HDFS', 'Linux', 'HealthApp', 'OpenStack', 'OpenSSH', 'Proxifier', 'HPC', 'Zookeeper', 'Mac', 'Hadoop', 'Android', 'Windows', 'Apache', 'Thunderbird', 'Spark']

datasets = ['HealthApp']

for dataset in datasets:
    print('-' * 50)
    print(f'Clustering {dataset} dataset...')

    # load the dataset
    df = pd.read_csv(f'dataset/{dataset}/{dataset}_2k.log_structured_corrected.csv')
    logs = df['Content'].tolist()
    templates = df['EventTemplate'].tolist()

    # tokenize -> vectorize -> cluster -> reassign_clusters
    tokenized_logs = [tokenize(log) for log in logs]
    print(vectorize(tokenized_logs))
    labels, cluster_nums = cluster(vectorize(tokenized_logs))
    labels, cluster_nums = reassign_clusters(labels, cluster_nums, tokenized_logs)

    # store the logs in the cluster
    inputs = []
    for i in range(cluster_nums):
        inputs.append([-1, [], [], '']) # label, logs, indexs, ground_truth
    for i, label in enumerate(labels):
        inputs[label][0] = label
        inputs[label][1].append(logs[i])
        inputs[label][2].append(i)
        if inputs[label][3] == '':
            inputs[label][3] = df['EventTemplate'][i]

    clusters = []
    for input in inputs:
        c = Cluster(*input, remove_duplicate= True)
        clusters.append(c)
    Count = 0
    for c in clusters:    
        if len(c.indexs) > 10:
            print(c.logs[0])
            print(c.oracle_template)
            print(len(c.indexs))
            print('=' * 50)
            Count+=1
    print(f'Count : {Count}')

## Show Clusters

In [ ]:
print('num of clusters:', cluster_nums)
print('len of templates:', len(set(templates)))

# store the logs in the cluster
inputs = []
for i in range(cluster_nums):
    inputs.append([-1, [], [], '']) # label, logs, indexs, ground_truth
for i, label in enumerate(labels):
    inputs[label][0] = label
    inputs[label][1].append(logs[i])
    inputs[label][2].append(i)
    if inputs[label][3] == '':
        inputs[label][3] = df['EventTemplate'][i]

clusters = []
for input in inputs:
    c = Cluster(*input, remove_duplicate= True)
    clusters.append(c)

# for cluster in clusters:
#     print(f'cluster {cluster.label}: {len(cluster.logs)} logs, {len(cluster.indexs)} indexs')

# num = 27
# print('cluster:', num)
# print('length:', len(clusters[num].indexs))
# print('template:', clusters[num].oracle_template)
# print('len of set:', len(clusters[num].logs))
# print('-'*20)
# for log in clusters[num].logs:
#     print(log)
# print('='*40)

# for cluster in clusters:
#     if len(set(cluster.logs)) == 1 and not any(char.isdigit() for char in cluster.logs[0]):
#         print(f"{cluster.logs[0]}\n{cluster.oracle_template}")
#         print('='*40)

## Evaluate

In [26]:
from evaluate import evaluate_all_datasets
from IPython.display import HTML

table = evaluate_all_datasets(
    'LogBatcher_0shot_32candidate_10batchsize_meta-llama_Llama-3-70b-chat-hf', send_email=False)
HTML(table)

        HDFS: group Accuracy: 1.0000, Message-Level Accuracy: 1.0000, Edit Distance: 0.0000, Normalized Edit Distance: 1.000000
      Hadoop: group Accuracy: 0.9910, Message-Level Accuracy: 0.9610, Edit Distance: 1.0850, Normalized Edit Distance: 0.988421
       Spark: group Accuracy: 0.9245, Message-Level Accuracy: 0.8825, Edit Distance: 1.8290, Normalized Edit Distance: 0.970398
   Zookeeper: group Accuracy: 0.9885, Message-Level Accuracy: 0.9875, Edit Distance: 0.1705, Normalized Edit Distance: 0.997280
         BGL: group Accuracy: 0.9950, Message-Level Accuracy: 0.9645, Edit Distance: 1.7440, Normalized Edit Distance: 0.989579
         HPC: group Accuracy: 0.9525, Message-Level Accuracy: 0.9265, Edit Distance: 0.6450, Normalized Edit Distance: 0.992882
 Thunderbird: group Accuracy: 0.7045, Message-Level Accuracy: 0.9230, Edit Distance: 0.7875, Normalized Edit Distance: 0.985225
     Windows: group Accuracy: 0.6915, Message-Level Accuracy: 0.4540, Edit Distance: 12.1435, Normalized

dataset,GA,PA,ED,N_ED
HDFS,1.0000,1.0000,0.0000,1.00000
Hadoop,0.9910,0.9610,1.0850,0.98842
Spark,0.9245,0.8825,1.8290,0.97040
Zookeeper,0.9885,0.9875,0.1705,0.99728
BGL,0.9950,0.9645,1.7440,0.98958
HPC,0.9525,0.9265,0.6450,0.99288
Thunderbird,0.7045,0.9230,0.7875,0.98522
Windows,0.6915,0.4540,12.1435,0.80646
Linux,0.9400,0.9225,0.5845,0.97843
Android,0.9730,0.7925,3.7485,0.95746


In [ ]:
from evaluate import evaluate_single_dataset
datasets = ['BGL', 'HDFS', 'HealthApp', 'OpenStack', 'OpenSSH', 'HPC', 'Zookeeper']
for dataset in datasets:
    evaluate_single_dataset(
        f'outputs/parser/Test_10shot_with_pruning/{dataset}_2k.log_structured.csv', dataset)

## Check out some strings' freq in the whole logs or templates

In [ ]:
import pandas as pd
from utils.cluster import tokenize
from utils.sample_byword import extract_variables
from utils.postprocess import correct_single_template
from utils.postprocess import extract_variables

datasets = ['BGL', 'HDFS', 'Linux', 'HealthApp', 'OpenStack', 'OpenSSH', 'HPC', 'Zookeeper', 'Mac', 'Hadoop', 'Android', 'Windows', 'Apache', 'Thunderbird', 'Spark']

count = 0
count_templates = []
for dataset in datasets:
    print(f"Processing {dataset} ----------------")
    df = pd.read_csv(f'dataset/{dataset}/{dataset}_2k.log_structured_corrected.csv')
    logs = df['Content'].tolist()
    templates = df['EventTemplate'].tolist()
    # templates = list(set(templates))
    for log, template in zip(logs, templates):
        if template not in count_templates:
            count_templates.append(template)
            count += 1
print(count)

## sample based on entropy

In [ ]:
from utils.sample import sample_based_on_entropy

# datasets = ['BGL', 'HDFS', 'Linux', 'HealthApp', 'OpenStack', 'OpenSSH', 'Proxifier', 'HPC', 'Zookeeper', 'Mac',
#         'Hadoop', 'Android', 'Windows', 'Apache', 'Thunderbird', 'Spark']
# for dataset in datasets:
#     pair = sample_based_on_entropy(dataset, 1)
#     print(pair[0][0])
dataset = 'HDFS'
pairs = sample_based_on_entropy(dataset, 1)
for pair in pairs:
    print(f"{pair[0][0]}\n{pair[0][1]}\n{'-'*20}")

## Token Caclulate

In [ ]:
import json
import tiktoken
import pandas as pd


def count_prompt_tokens(prompt, model_name):
    # 根据模型名称加载合适的编码器
    if model_name == "gpt-4":
        encoder = tiktoken.encoding_for_model("gpt-4")
    elif model_name == "gpt-3.5-turbo":
        encoder = tiktoken.encoding_for_model("gpt-3.5-turbo")
    else:
        raise ValueError("未知的模型名称")

    # 计算编码后的token数
    prompt_tokens = encoder.encode(prompt)
    return len(prompt_tokens)


def count_message_tokens(messages, model_name):
    # 根据模型名称加载合适的编码器
    if model_name == "gpt-4":
        encoder = tiktoken.encoding_for_model("gpt-4")
    elif model_name == "gpt-3.5-turbo":
        encoder = tiktoken.encoding_for_model("gpt-3.5-turbo")
    else:
        raise ValueError("未知的模型名称")

    # 初始化token计数
    token_count = 0

    # 计算每个消息的token数
    for message in messages:
        role_tokens = encoder.encode(message['role'])
        content_tokens = encoder.encode(message['content'])
        token_count += len(role_tokens) + \
            len(content_tokens) + 4  # 加上特殊的消息分隔符的token数

    return token_count

## Caculate Cost

In [ ]:



table_order = 'HDFS Hadoop Spark Zookeeper BGL HPC Thunderbird Windows Linux Android HealthApp Apache OpenSSH OpenStack Mac'
datasets = table_order.split(' ')
logs = {}
counts_token = {}
counts_message = {}
for dataset in datasets:
    df = pd.read_csv(f'dataset/{dataset}/{dataset}_2k.log_structured_corrected.csv')
    logs[dataset] = df['Content'].tolist()
    counts_token[dataset] = 0
    counts_message[dataset] = 0
for log in logs['HealthApp']:
    log = log.strip()

# 存储解析后的日志列表
message_list = []
# load every message
file = 'cost_lilac_32_3.json'
with open('outputs/cost/LogBatcher_3shot_32candidate_10batchsize.json', 'r') as file:
    for line in file:
        if line.strip() == '[':
            list_str = ''
            start_load = True
        if line.strip() == ']':
            list_str += line
            message = json.loads(list_str)
            message_list.append(message)
            start_load = False
        if start_load:
            list_str += line
# print(len(message_list))
for message in message_list:
    # for LILAC
    log = message[-1]['content'].split('\n')[0].replace('Log message: `', '').replace('`', '')
    # for LogBatcher
    # log = message[-1]['content'].split('\n')[0] 
    # print(log)
    for dataset in datasets:
        if log in logs[dataset]:
            counts_token[dataset] += count_message_tokens(message, 'gpt-3.5-turbo')
            counts_message[dataset] += 1
            break
        if dataset == 'Mac':
            print(log)
for dataset in datasets:
    print(counts_token[dataset], counts_token[dataset] / counts_message[dataset]  )

print(sum(counts_token.values()), sum(counts_token.values()) / sum(counts_message.values()))

# remove the same log messages

# def make_hashable(log_list):

#     return tuple(tuple(sorted(d.items())) for d in log_list)
# unique_lists = list(set(make_hashable(log_list) for log_list in message_list))

# unique_big_list = [list(map(dict, log_list)) for log_list in unique_lists]
# print(len(unique_big_list))

In [19]:


table_order = 'HDFS Hadoop Spark Zookeeper BGL HPC Thunderbird Windows Linux Android HealthApp Apache OpenSSH OpenStack Mac'
datasets = table_order.split(' ')
# datasets = ['HDFS']
logs = {}
counts_token = {}
counts_message = {}
for dataset in datasets:
    counts_token[dataset] = 0
    counts_message[dataset] = 0
    with open(f'DivLog/cost/cost_divlog_for_{dataset}.json', 'r') as file:
        prompt_list = json.load(file)
    # print(f"caculate {dataset}: len: {len(prompt_list)}")
    for prompt in prompt_list:
        counts_token[dataset] += count_prompt_tokens(prompt, 'gpt-3.5-turbo')
        counts_message[dataset] += 1
    print(counts_token[dataset], counts_token[dataset] / counts_message[dataset])

# print average token count
print(sum(counts_token.values()), sum(counts_token.values()) / sum(counts_message.values()))

353.3445
245.6495
204.527
180.279
210.173
144.477
261.2915
230.9235
222.2085
215.33
175.8405
182.304
245.3635
343.0185
290.645


TypeError: 'int' object is not callable

In [20]:
print(count_prompt_tokens('For each log after <prompt> tag, extract one log template(substitute variable tokens in the log as <*> and remain constant tokens to construct the template)and put the template after <extraction> tag and between <START> and <END> tags.', 'gpt-3.5-turbo'))

52


In [21]:
from utils.sample_byword import extract_variables
print(extract_variables("488205 floating point alignment exceptions",
      "<*> floating point alignment exceptions")) 

('488205',)


In [23]:
list1 = [1,2,3,4,5]
class test:
    def __init__(self):
        self.list2 = [1,2,3,4,7]

t_class = test()
for a in list1:
    if a == 4:
        t_class.list2.remove(a)
print(t_class.list2)
print(list1)

[1, 2, 3, 7]
[1, 2, 3, 4, 5]
